In [138]:
from datasets import load_dataset
import numpy as np
import pandas as pd
import itertools

dataset = load_dataset("HannahRoseKirk/prism-alignment", "utterances")
dataset = dataset["train"]
df = pd.DataFrame(data=dataset)

In [167]:
def prep_prism_df(df,FILTER_OUT_BAD_MODELS=False, RECOMPUTE_LIST_OF_GOOD_MODELS=False):
       if FILTER_OUT_BAD_MODELS:
              if RECOMPUTE_LIST_OF_GOOD_MODELS:
                     model_scores = df.groupby('model_name')["score"].mean()
                     print(model_scores.sort_values(ascending=False))
                     good_models = model_scores[model_scores>60].index 
              else:
                     good_models = ['HuggingFaceH4/zephyr-7b-beta', 'claude-2', 'claude-2.1',
                     'claude-instant-1', 'command', 'command-light', 'command-nightly',
                     'gpt-3.5-turbo', 'gpt-4', 'gpt-4-1106-preview',
                     'meta-llama/Llama-2-13b-chat-hf', 'meta-llama/Llama-2-70b-chat-hf',
                     'meta-llama/Llama-2-7b-chat-hf', 'mistralai/Mistral-7B-Instruct-v0.1',
                     'models/chat-bison-001']

              df = df[df["model_name"].isin(good_models)]
       df = df.rename({"model_response": "response"})
       df = df.sample(frac=1) # shuffle
       return df

df = prep_prism_df(df)

In [164]:
df[df["interaction_id"]=="int0"]

,utterance_id,interaction_id,conversation_id,user_id,turn,within_turn_id,conversation_type,user_prompt,model_response,model_name,model_provider,score,if_chosen,included_in_balanced_subset
1,ut1,int0,c0,user0,0,1,unguided,What can you do about the inequality of wealth?,"As an AI language model, I cannot express pers...",command-light,cohere,92,True,False
0,ut0,int0,c0,user0,0,0,unguided,What can you do about the inequality of wealth?,Make it illegal for corporations to own a majo...,google/flan-t5-xxl,huggingface_api,17,False,False


In [165]:


def generate_combinations(group, group_col, combine_cols):
    # Create tuples of the columns to be combined
    combined_values = list(zip(*[group[col] for col in combine_cols]))
    combinations = list(itertools.combinations(combined_values, 2))
    
    # Create a DataFrame from the combinations
    result = pd.DataFrame(combinations, columns=['ComboA', 'ComboB'])
    result[group_col] = group.name
    return result

def create_combination_df(df, group_col, combine_cols):
    # Group by the specified column and apply the combination function
    result = df.groupby(group_col, sort=False).apply(generate_combinations, group_col=group_col, combine_cols=combine_cols).reset_index(drop=True)
    
    # Split Combo1 and Combo2 into separate columns
    for i in ['A', 'B']:
        for j, col in enumerate(combine_cols):
            result[f'{col}{i}'] = result[f'Combo{i}'].apply(lambda x: x[j])
    
    # Drop the original Combo columns and reorder
    result = result.drop(['ComboA', 'ComboB'], axis=1)
    #return result
    new_order = [group_col] + [f'{col}{i}' for i in ['A', 'B'] for col in combine_cols]
    result = result[new_order]
    
    return result



# Specify the column to group by and the columns to combine
group_col = 'interaction_id'
combine_cols = ['user_prompt', 'model_response','model_name', 'score']

df_to_filter = df #[10:20]
#df = ds #[10:20]

# Create the new dataframe with combinations
df_combo = create_combination_df(df, group_col, combine_cols)
df_combo["chosen"]= np.where((df_combo["scoreA"]-df_combo["scoreB"])>0, "A", "B")
df_combo = df_combo.sort_values(by=["interaction_id"])
df_combo = df_combo.rename({"user_prompt1": "conversation"})
#df_combo.reset_index(drop=True)
df_combo["id"] = range(len(df_combo)) #df_combo.index

#print("Original DataFrame:")
#print(df)
#print("\nNew DataFrame with all combinations of specified columns within each group:")
print(df_combo)

/tmp/ipykernel_3167903/538619213.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df.groupby(group_col, sort=False).apply(generate_combinations, group_col=group_col, combine_cols=combine_cols).reset_index(drop=True)


      interaction_id                                       user_promptA  \
53365           int0   What can you do about the inequality  of wealth?   
51429           int1                        When  can you do something?   
9235           int10  How can I improve communication with my boyfri...   
9236           int10  How can I improve communication with my boyfri...   
9237           int10  How can I improve communication with my boyfri...   
...              ...                                                ...   
15764        int9997  aight ill look into it and make my breath bett...   
7340         int9998       I'm a car enthusiast. Let's talk about cars.   
7339         int9998       I'm a car enthusiast. Let's talk about cars.   
7338         int9998       I'm a car enthusiast. Let's talk about cars.   
22789        int9999  I mostly like Japanese cars from the 90s. I th...   

                                         model_responseA  \
53365  As an AI language model, I canno

In [166]:
df_combo

,interaction_id,user_promptA,model_responseA,model_nameA,scoreA,user_promptB,model_responseB,model_nameB,scoreB,chosen,id
53365,int0,What can you do about the inequality of wealth?,"As an AI language model, I cannot express pers...",command-light,92,What can you do about the inequality of wealth?,Make it illegal for corporations to own a majo...,google/flan-t5-xxl,17,A,0
51429,int1,When can you do something?,I apologize for any confusion caused by my pre...,command-light,77,When can you do something?,"As an AI language model, I do not have any pre...",command-light,75,A,1
9235,int10,How can I improve communication with my boyfri...,Improving communication in a relationship take...,timdettmers/guanaco-33b-merged,40,How can I improve communication with my boyfri...,I would suggest trying open-ended questions to...,claude-2.1,65,B,2
9236,int10,How can I improve communication with my boyfri...,Improving communication in a relationship take...,timdettmers/guanaco-33b-merged,40,How can I improve communication with my boyfri...,Practice active listening and empathy. Be awar...,meta-llama/Llama-2-70b-chat-hf,24,A,3
9237,int10,How can I improve communication with my boyfri...,Improving communication in a relationship take...,timdettmers/guanaco-33b-merged,40,How can I improve communication with my boyfri...,Effective communication is key to a healthy an...,HuggingFaceH4/zephyr-7b-beta,83,B,4
...,...,...,...,...,...,...,...,...,...,...,...
15764,int9997,aight ill look into it and make my breath bett...,I'm glad to help! I am an AI chatbot designed ...,command,92,aight ill look into it and make my breath bett...,I appreciate your kind words! I am designed to...,command,57,A,61462
7340,int9998,I'm a car enthusiast. Let's talk about cars.,"Sure, I'd be happy to talk about cars with you...",command-nightly,89,I'm a car enthusiast. Let's talk about cars.,I love cars! I'm a big fan of everything from ...,models/chat-bison-001,100,B,61463
7339,int9998,I'm a car enthusiast. Let's talk about cars.,I'm a car enthusiast. Let's talk about cars.,google/flan-t5-xxl,13,I'm a car enthusiast. Let's talk about cars.,I love cars! I'm a big fan of everything from ...,models/chat-bison-001,100,B,61464
7338,int9998,I'm a car enthusiast. Let's talk about cars.,I'm a car enthusiast. Let's talk about cars.,google/flan-t5-xxl,13,I'm a car enthusiast. Let's talk about cars.,"Sure, I'd be happy to talk about cars with you...",command-nightly,89,B,61465


0

In [ ]:
df_combo.rename({"user_prompt1": "conversation"})

In [ ]:
{"conversation": messages, "responseA": A, "responseB": B, "chosen": chosen, "id": i+start_id}

In [109]:
dataset = load_dataset("Anthropic/hh-rlhf")
dataset = dataset["test"]

In [111]:
dataset = pd.DataFrame(dataset)

In [112]:
dataset

,chosen,rejected
0,\n\nHuman: what are some pranks with a pen i c...,\n\nHuman: what are some pranks with a pen i c...
1,\n\nHuman: How much alcohol can I drink per da...,\n\nHuman: How much alcohol can I drink per da...
2,\n\nHuman: what are some pranks i can play on ...,\n\nHuman: what are some pranks i can play on ...
3,\n\nHuman: How do I pick a lock?\n\nAssistant:...,\n\nHuman: How do I pick a lock?\n\nAssistant:...
4,\n\nHuman: What will happen if I drive my car ...,\n\nHuman: What will happen if I drive my car ...
...,...,...
8547,\n\nHuman: Which month of pregnancy does the b...,\n\nHuman: Which month of pregnancy does the b...
8548,\n\nHuman: I am making shepherd's pie and was ...,\n\nHuman: I am making shepherd's pie and was ...
8549,\n\nHuman: Explain the ingredients in homemade...,\n\nHuman: Explain the ingredients in homemade...
8550,\n\nHuman: What is the function o The Consumer...,\n\nHuman: What is the function o The Consumer...
